<a href="https://colab.research.google.com/github/RemyLpr/defi_ia/blob/main/model/Model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import des modules et des données

In [1]:
import pandas as pd
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFECV
import warnings
warnings.filterwarnings('ignore')

In [33]:
df = pd.read_csv("train_set.csv") # données d'entraînement
df_eval = pd.read_csv("test_set.csv") # données de test du site
df_hotel = pd.read_csv("features_hotels.csv") # données sur les hôtels
df_clust = pd.read_csv("hotel_price_clust.csv") # clusters sur les prix des hôtels à J44
df_stock_j44_clust = pd.read_csv("stock_j44_clust.csv") # clusters sur les stocks des hôtels à J44
df_golden = pd.read_csv("golden_vente_local.csv") # feature golden
df_lang = pd.read_csv("coeff_lang.csv") # coefficients pour chaque langue

# Preview des données

In [5]:
df

,hotel_id,price,stock,city,date,language,mobile,avatar_id,avatar_name,order_requests
0,768,286,13,madrid,11,portuguese,0,155051,8HOAWS,1
1,126,96,8,madrid,11,portuguese,0,155051,8HOAWS,1
2,378,124,0,madrid,11,portuguese,0,155051,8HOAWS,1
3,351,91,4,madrid,11,portuguese,0,155051,8HOAWS,1
4,747,172,15,madrid,11,portuguese,0,155051,8HOAWS,1
...,...,...,...,...,...,...,...,...,...,...
166832,503,277,4,vilnius,5,lithuanian,0,267714,9HJM3R,1566
166833,240,224,4,vilnius,5,lithuanian,0,267714,9HJM3R,1566
166834,820,207,4,vilnius,5,lithuanian,0,267714,9HJM3R,1566
166835,288,113,0,vilnius,5,lithuanian,0,267714,9HJM3R,1566


In [6]:
df_eval

,index,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock
0,0,1,vilnius,21,romanian,0,1,161,46
1,1,1,vilnius,21,romanian,0,1,187,32
2,2,1,vilnius,21,romanian,0,1,279,12
3,3,1,vilnius,21,romanian,0,1,395,10
4,4,1,vilnius,21,romanian,0,1,488,42
...,...,...,...,...,...,...,...,...,...
6639,6639,843,rome,5,irish,0,794,987,1
6640,6640,844,vienna,1,irish,1,794,26,1
6641,6641,844,vienna,1,irish,1,794,263,0
6642,6642,844,vienna,1,irish,1,794,456,0


# Préparation des données

Tout d'abord on homogénéise les colonnes (variables d'entrée des datasets de train et de test). On fait aussi la jointure avec le dataframe contenant les features des hôtels.

In [34]:
df.drop("avatar_name", axis = 1, inplace = True)
df_eval.drop("index", axis = 1, inplace = True)
df_hotel.drop("city", axis = 1, inplace = True) # on doit la supprimer immédiatement pour ne pas créer de doublon
df = pd.merge(df, df_hotel, on = "hotel_id", how = "left") # on merge avec le csv features_hotels
df_eval = pd.merge(df_eval, df_hotel, on = "hotel_id", how = "left")
df = pd.merge(df, df_clust, on = "hotel_id", how = "left") # on merge avec le csv hotel_price_clust
df_eval = pd.merge(df_eval, df_clust, on = "hotel_id", how = "left")
df = pd.merge(df, df_stock_j44_clust, on = "hotel_id", how = "left") # on merge avec le csv stock_j44_clust
df_eval = pd.merge(df_eval, df_stock_j44_clust, on = "hotel_id", how = "left")
mobile_list_1, mobile_list_2 = [], []

Le merge avec la feature stagne, qui varie selon la ville ET le jour, est un peu différent et nécessite un traitement à part. Aussi, un comportement étrange a été remarqué pour la ville Madrid au jour 2, pour certains hôtels la valeur de la variable mobile a une influence sur le prix, contrairement à habituellement. On va donc détecter quels sont ces hôtels et les stocker dans des listes pour modifier leur prix après.

In [35]:
# ATTENTION : cellule à réexécuter après création des mobile_list
golden_list = []
for index, row in df.iterrows():
  city, hotel, date, mobile = row["city"], row["hotel_id"], row["date"], row["mobile"]
  if (date==2) & (mobile==1) & (hotel in mobile_list_2): # cas particulier repéré où la valeur de mobile a une influence pour la date 2
    golden_list.append(round(0.92*df_golden.iloc[hotel][date]))
  else:
    if (date==1) & (mobile==1) & (hotel in mobile_list_1): # pareil pour la date 1
      golden_list.append(round(0.92*df_golden.iloc[hotel][date]))
    else:
      golden_list.append(df_golden.iloc[hotel][date])
df["golden"] = golden_list
mobile_list_1, mobile_list_2 = [], [] # on vide les listes pour éviter des problèmes lors de potentielles réexécutions

In [36]:
golden_list = []
for index, row in df_eval.iterrows():
  hotel, date = row["hotel_id"], row["date"]
  golden_list.append(df_golden.iloc[hotel][date])
df_eval["golden"] = golden_list

In [37]:
golden_list = []
# ATTENTION : cellule à réexécuter après création des mobile_list
for index, row in df.iterrows():
  language, city, golden = row["language"], row["city"], row["golden"]
  coeff = df_lang[(df_lang["language"] == "french") & (df_lang["city"] == "paris")]["coeff"].values.tolist()[0]
  golden_list.append(coeff * golden)
df["golden"] = golden_list

In [18]:
row_list = []
a = df[df["language"] == "hungarian"]
for index, row in a.iterrows():
  if abs(row["price"] - row["stagne"]) > 1:
    row_list.append(row)
df_errors = pd.DataFrame(data=row_list)
df_errors_1, df_errors_2 = df_errors[df_errors["date"] == 1], df_errors[df_errors["date"] == 2]
mobile_list_1, mobile_list_2 = df_errors_1["hotel_id"].tolist(), df_errors_2["hotel_id"].tolist()

Pour l'instant on va supprimer les colonnes avatar_id, hotel_id et order_requests pour se faciliter la tâche. De nouvelles variables pourraient être générées à partir de celles-ci mais pour l'instant notre but est juste de faire tourner un premier modèle.

In [ ]:
df.drop(["avatar_id", "hotel_id", "order_requests"], axis = 1, inplace = True)
df_eval.drop(["avatar_id", "hotel_id", "order_requests"], axis = 1, inplace= True)

Ensuite on trie les colonnes par ordre alphabétique pour une visualisation/comparaison plus claire de celles-ci.

In [ ]:
df = df.reindex(sorted(df.columns), axis=1)
df_eval = df_eval.reindex(sorted(df_eval.columns), axis=1)

On doit encoder les variables catégorielles "city" et "language". Pour effectuer cette tâche nous allons utliser la méthode get_dummies de pandas car les modalités de ces variables n'ont pas de lien entre elles.

In [ ]:
# génération des dummies pour les variables city, language, brand et group
cities = pd.get_dummies(df["city"], prefix_sep = "_", prefix = "city")
languages =  pd.get_dummies(df["language"], prefix_sep = "_", prefix = "language")
brands = pd.get_dummies(df["brand"], prefix_sep = "_", prefix = "brand")
groups = pd.get_dummies(df["group"], prefix_sep = "_", prefix = "group")
df = pd.concat([df, cities, languages, brands, groups], axis = 1) # on ajoute les colonnes générées au df
df.drop(["city", "language", "brand", "group"], axis = 1, inplace = True) # on supprime les colonnes d'origine

On effectue le même processus pour le dataset d'évaluation.

In [ ]:
cities = pd.get_dummies(df_eval["city"], prefix_sep = "_", prefix = "city")
languages =  pd.get_dummies(df_eval["language"], prefix_sep = "_", prefix = "language")
brands = pd.get_dummies(df_eval["brand"], prefix_sep = "_", prefix = "brand")
groups = pd.get_dummies(df_eval["group"], prefix_sep = "_", prefix = "group")
df_eval = pd.concat([df_eval, cities, languages, brands, groups], axis = 1)
df_eval.drop(["city", "language", "brand", "group"], axis = 1, inplace = True)

## Train test split

Maintenant on sépare la variable de sortie (price) des variables d'entrée.

In [ ]:
X = df.drop("price", axis = 1)
y = df["price"]

Enfin, on split nos données en entraînement/test (répartition 80/20).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Scaling des données

Partie non obligatoire : scaler les données pour permettre une meilleure prédiction du modèle. N'est pas nécessaire pour tous les modèles.

In [ ]:
scaler = StandardScaler()
X[['cluster_hotel','date', 'stock', 'stock_j44_clust']] = scaler.fit_transform(df[['cluster_hotel','date', 'stock', 'stock_j44_clust']]) 
df_eval[['cluster_hotel','date', 'stock', 'stock_j44_clust']] = scaler.fit_transform(df_eval[['cluster_hotel','date', 'stock', 'stock_j44_clust']])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Entraînement et test d'un modèle

Ici je vaiis utiliser un modèle de type MLPRegressor [https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor](https://).
Nous allons déclarer le modèle puis l'entraîner et le tester.

In [ ]:
# import du régresseur
from sklearn.neural_network import MLPRegressor
# définition du régresseur utilisé et entraînement de celui-ci
reg = MLPRegressor(hidden_layer_sizes = (256, 256, 256), verbose = True, early_stopping = True, tol = 0.05, n_iter_no_change=1, validation_fraction=0.1).fit(X_train, y_train)

Iteration 1, loss = 104.94022182
Validation score: 0.998447


In [ ]:
y_pred = reg.predict(X_test)
rmse = mean_squared_error(y_true = y_test, y_pred = y_pred, squared = False)
print('rmse : ', rmse)

rmse :  3.1829956792531617


# Export des résultats et évaluation sur Kaggle

On doit ensuite utiliser le dataset de test fourni par sur le Kaggle en renvoyer le fichier au format demandé.

In [ ]:
eval = reg.predict(df_eval)
submission = pd.DataFrame(data = eval.tolist(), columns = ["price"])
submission.to_csv("MLP_first_iteration.csv", index_label='index') # tout ça pour créer la colonne Unnamed et la transformer en index (nécessaire pour l'évaluation par le site)